# Add external catalog for source matching:

This notebook demonstrate how to add different types of astronomical catalogs into a NoSql database for later use. The process of adding an external catalog has the following steps:

1. **Ingestion**: a database is created containing the catalog. Indexes are built as desired.
2. **Testing**: a few basic tests are run on the database, to measure query execution times and catch potential errors.
3. **Documenting**: a few key information have to be added to the catalog, such as the keys and method that can be used to support queries. Description of the catalog, and references can also be added.

The base class to perform these operation is the CatalogPusher class. In the following we will make some examples, trying to illustrate the basic options of the code. For this purpose will will make use of the Million Quasr Catalog (milliquas).

The milliquas catalog is a compendium of ~ 2M QSOs and AGN, largely complete from the literature up to 5 August 2017. It includes data from SDSS-DR14 NBCKDE, NBCKDE-v3, XDQSO, AllWISE and Peters photometric quasar catalogs and from all-sky radio/X-ray associated objects which are calculated by the author. Each object is presented with its original name, best redshift, and 0.1-arcsecond-accurate astrometry (sourced from optical APM/USNO-B/SDSS data). Ref: https://heasarc.gsfc.nasa.gov/W3Browse/all/milliquas.html

The data used for this test example notebook can be downloaded from: https://desycloud.desy.de/index.php/s/0KiEujzQ4yN1lVQ

## 1) Inserting:

While the CatalogPusher class provides interface to common/bulk operations, as catalogs come in a wide variety of sizes and formats, the user is requested to contribute two key functions to allow the external catalog to be parsed and ingested in the databse:

* a 'reader' to parse the raw files with the catalog data into a list of dictionaries
* a 'modifier' to edit the document of the single sources (add healpix indexes, format coordinates, remove columns, ecc.)

In [1]:
from extcats import CatalogPusher

# build the pusher object and point it to the raw files.
mqp = CatalogPusher.CatalogPusher(
    catalog_name = 'milliquas',             # short name of the catalog
    data_source = '../testdata/milliquas/', # where to find the data
    file_type = 'tdat.gz'
    )


# define the reader for the raw files. In this case the formatting or the raw file 
# is pretty ugly so we have to put quite a lot of information here.
# the pandas package provides very efficient ways to read flat files and its use
# is recommended. For this specific example see:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html
catcols=['name', 'ra', 'dec', 'lii', 'bii', 'broad_type', 'rmag', 
         'bmag', 'optical_flag', 'red_psf_flag', 'blue_psf_flag',
         'redshift', 'ref_name', 'ref_redshift', 'qso_prob', 
         'radio_name', 'xray_name', 'alt_name_1', 'alt_name_2', 'class']
import pandas as pd
mqp.assign_file_reader(
        reader_func = pd.read_table,         # callable to use to read the raw_files. 
        read_chunks = True,                  # weather or not the reader process each file into smaller chunks.
        names=catcols,                       # All other arguments are passed directly to this function.
        chunksize=50000,
        engine='c',
        skiprows=65,
        sep='|',
        index_col=False,
        comment='<')


# now we have to define a modifier function that acts on the single documents
# (dictionaries) and format them in the way they have to appear in the database.
# in this case we format coordinates in the geoJSON type (this enables mongo to
# support queries in spherical cooridnates), and we assign to each source its
# healpix index on a grid of order 16, corresponding to ~3" resolution.
from healpy import ang2pix
def mqc_modifier(srcdict):
    
    # format coordinates into geoJSON type (commented version uses 'legacy' pair):
    # unfortunately mongo needs the RA to be folded into -180, +180
    ra=srcdict['ra'] if srcdict['ra']<180. else srcdict['ra']-360.
    srcdict['pos']={
            'type': 'Point', 
            'coordinates': [ra, srcdict['dec']]
                    }
    #srcdict['pos']=[srcdict['ra'], srcdict['dec']] # This is the legacy coordinate format
    
    # add healpix index
    srcdict['hpxid_16']=int(
        ang2pix(2**16, srcdict['ra'], srcdict['dec'], lonlat = True, nest = True))
    
    return srcdict

mqp.assign_dict_modifier(mqc_modifier)

# fill in the database, creting indexes on the position and healpix ID.
import pymongo
mqp.push_to_db(
    coll_name = 'srcs', 
    index_on = ['hpxid_16', [('pos', pymongo.GEOSPHERE)] ] ,
    index_args = [{}, {}], # specify arguments for the index creation
    overwrite_coll = False, 
    append_to_coll = False)

# now print some info on the database
#mqp.info()

INFO:extcats.CatalogPusher:found 1 files for catalog milliquas in data source: ['../testdata/milliquas/']
INFO:extcats.CatalogPusher:checking raw files for existence and consistency..
INFO:extcats.CatalogPusher:all files exists and have consistent type.
INFO:extcats.CatalogPusher:file reader read_table assigned to pusher.
INFO:extcats.CatalogPusher:source document modifer mqc_modifier assigned to pusher.
INFO:extcats.CatalogPusher:using mongo client at localhost:27017
INFO:extcats.CatalogPusher:connecting to database milliquas. Here some stats:
INFO:extcats.CatalogPusher:{
  "db": "milliquas",
  "collections": 2,
  "views": 0,
  "objects": 1998468,
  "avgObjSize": 470.90632974858744,
  "dataSize": 941091231.0,
  "storageSize": 287059968.0,
  "numExtents": 0,
  "indexes": 4,
  "indexSize": 87138304.0,
  "fsUsedSize": 194300112896.0,
  "fsTotalSize": 231446335488.0,
  "ok": 1.0
}


## 2) Testing the catalog

At this stage, a simple test is run on the database, consisting in crossmatching with a set of randomly distributed points. The query time is also measured, allowing to decide on the indexing strategy and query functions.

The _run_test_ method of the CatalogPusher object provides convenient way to test a user defined query function on a set of uniformly ditributed points on a sphere. Standard query functions are defined in the CatalogQuery module. They are decribed in the _query_example_ notebook. 

In [2]:
# define the funtion to test coordinate based queries:
import numpy as np
from healpy import ang2pix, get_all_neighbours
from astropy.table import Table
from astropy.coordinates import SkyCoord
from math import radians
# define your search radius
rs_arcsec = 10.

def test_query_healpix(ra, dec, coll):
    """query collection for the closest point within 
    rs_arcsec of target ra, dec. It uses healpix ID
    to perform the search.
    
    The results as returned as an astropy Table. """
    
    # find the index of the target pixel and its neighbours 
    target_pix = int( ang2pix(2**16, ra, dec, nest = True, lonlat = True) )
    neighbs = get_all_neighbours(2*16, ra, dec, nest = True, lonlat = True)

    # remove non-existing neigbours (in case of E/W/N/S) and add center pixel
    pix_group = [int(pix_id) for pix_id in neighbs if pix_id != -1] + [target_pix]
    
    # query the database for sources in these pixels
    qfilter = { 'hpxid_16': { '$in': pix_group } }
    qresults = [o for o in coll.find(qfilter)]
    if len(qresults)==0:
        return None
    
    # then use astropy to find the closest match
    tab = Table(qresults)
    target = SkyCoord(ra, dec, unit = 'deg')
    matches_pos = SkyCoord(tab['ra'], tab['dec'], unit = 'deg')
    d2t = target.separation(matches_pos).arcsecond
    match_id = np.argmin(d2t)

    # if it's too far away don't use it
    if d2t[match_id]>rs_arcsec:
        return None
    return tab[match_id]


def test_query_2dsphere(ra, dec, coll):
    """query collection for the closest point within 
    rs_arcsec of target ra, dec. It uses mondod spherical
    geometry queries.
    
    The results as returned as an astropy Table. """
    
    
    # fold the RA between -180 and 180.
    if ra > 180:
        ra = ra - 360.
    
    # query and return
    geowithin={"$geoWithin": { "$centerSphere": [[ra, dec], radians(rs_arcsec/3600.)]}}
    qresults = [o for o in coll.find({"pos": geowithin})]
    if len(qresults)==0:
        return None
    
    # then use astropy to find the closest match
    tab = Table(qresults)
    target = SkyCoord(ra, dec, unit = 'deg')
    matches_pos = SkyCoord(tab['ra'], tab['dec'], unit = 'deg')
    d2t = target.separation(matches_pos).arcsecond
    match_id = np.argmin(d2t)

    # if it's too far away don't use it
    if d2t[match_id]>rs_arcsec:
        return None
    return tab[match_id]

# run the test. Here we compare queries based on the 
# healpix index with those based on the 2dsphere mongod support.
mqp.run_test(test_query_healpix, npoints = 1000)
mqp.run_test(test_query_2dsphere, npoints = 1000)

INFO:extcats.CatalogPusher:running test queries using 1000 random points
100%|██████████| 1000/1000 [00:00<00:00, 1835.14it/s]
INFO:extcats.CatalogPusher:Total document found for query: 0
INFO:extcats.CatalogPusher:Took 5.46e-01 sec for 1000 random queries. Average query time: 5.463e-04 sec
INFO:extcats.CatalogPusher:running test queries using 1000 random points
100%|██████████| 1000/1000 [00:00<00:00, 2052.31it/s]
INFO:extcats.CatalogPusher:Total document found for query: 0
INFO:extcats.CatalogPusher:Took 4.89e-01 sec for 1000 random queries. Average query time: 4.892e-04 sec


# 3) Adding metadata

Once the database is set up and the query performance are satisfactory, metadata describing the catalog content, contact person, and query strategies have to be added to the catalog database. If presents, the keys and parameters for the healpix partitioning of the sources are also to be given, as well as the name of the compound geoJSON/legacy pair entry in the documents.

This information will be added into the 'metadata' collection of the database which will be accessed by the CatalogQuery. The metadata will be stored in a dedicated collection so that the database containig a given catalog will have two collections:
    - db['srcs'] : contains the sources.
    - db['meta'] : describes the catalog.

In [6]:
mqp.healpix_meta(healpix_id_key = 'hpxid_16', order = 16, is_indexed = True, nest = True)
mqp.sphere2d_meta(sphere2d_key = 'pos', is_indexed = True, pos_format = 'geoJSON')
mqp.science_meta(
    contact =  'C. Norris', 
    email = 'chuck.norris@desy.de', 
    description = 'compilation of AGN and Quasar',
    reference = 'http://quasars.org/milliquas.htm')

INFO:extcats.CatalogPusher:adding metadata for HEALPix key named: hpxid_16
INFO:extcats.CatalogPusher:adding metadata for 2dsphere coordinates named: pos
INFO:extcats.CatalogPusher:adding science related metadata to database.
